In [7]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *

In [8]:
lenet = Lenet()
mnist = MNIST()
train_loader, test_loader = mnist.get_dataloader()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.001)

In [11]:
#for only training of any model (pruned or unpruned)
trainer=Trainer(lenet, 5, train_loader, criterion, optimizer)
trainer.train()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.9617 | 
Epoch: 2 | train_loss: 1.5355 | 
Epoch: 3 | train_loss: 1.2733 | 
Epoch: 4 | train_loss: 0.9565 | 
Epoch: 5 | train_loss: 0.7947 | 


Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [12]:
trainer = UnstructuredL1normPrune(lenet, 5, train_loader, criterion, optimizer, 0.9)
#train and prune the model and return both the original and pruned model
model, pruned_model = trainer.train_prune_retrain()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5576 | 
Epoch: 2 | train_loss: 0.2438 | 
Epoch: 3 | train_loss: -0.0241 | 
Epoch: 4 | train_loss: -0.2780 | 
Epoch: 5 | train_loss: -0.4594 | 
Training is done
Pruning is done


In [13]:
print(accuracy(model, test_loader))
print(accuracy(pruned_model, test_loader))

[0.9991]
[0.1661]


In [14]:
print(non_zero_weights(model))
print(non_zero_weights(pruned_model))
print(compression_ratio(model, pruned_model))

44190
4419
0.1
